In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
service_credential = dbutils.secrets.get(scope="databricksscopename",key="databricksservicekey")

spark.conf.set("fs.azure.account.auth.type.adlsx5u224gv3xgd2.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.adlsx5u224gv3xgd2.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.adlsx5u224gv3xgd2.dfs.core.windows.net", "5c962712-e111-48b0-b9af-41f03175e216")
spark.conf.set("fs.azure.account.oauth2.client.secret.adlsx5u224gv3xgd2.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.adlsx5u224gv3xgd2.dfs.core.windows.net", "https://login.microsoftonline.com/72f988bf-86f1-41af-91ab-2d7cd011db47/oauth2/token")

In [ ]:
customers_schema = "customer_id integer,tax_id STRING,tax_code STRING, customer_name STRING,state STRING,city STRING,postcode STRING,street STRING,number STRING,unit STRING,region STRING,district STRING,lon double,lat double,ship_to_address STRING,valid_from STRING,valid_to STRING,units_purchased STRING,loyalty_segment integer, customer_key BIGINT"

customers_df = spark.read \
        .format("delta") \
        .load("abfss://data@adlsx5u224gv3xgd2.dfs.core.windows.net/retail_org_dt/Silver/customers")

customers_df = customers_df.withColumn("customer_key", expr("monotonically_increasing_id() + 1"))

dim_customers = spark.createDataFrame(customers_df.rdd, customers_schema)

dim_customers.write.format("delta").option('path','abfss://data@adlsx5u224gv3xgd2.dfs.core.windows.net/retail_org_dt/Gold/customers').saveAsTable("retail_org_dt.dim_customers")

In [ ]:
products_schema = "product_id STRING,product_category STRING,product_name STRING,sales_price STRING,ean13 DOUBLE,ean5 STRING,product_unit STRING, product_key BIGINT"

products_df = spark.read \
        .format("delta") \
        .load("abfss://data@adlsx5u224gv3xgd2.dfs.core.windows.net/retail_org_dt/Silver/products") \
        .select("product_id", "product_category", "product_name", "sales_price", "ean13", "ean5", "product_unit")

products_df = products_df.withColumn("product_key", expr("monotonically_increasing_id() + 1"))

dim_products = spark.createDataFrame(products_df.rdd, products_schema)

dim_products.write.format("delta").option('path','abfss://data@adlsx5u224gv3xgd2.dfs.core.windows.net/retail_org_dt/Gold/products').saveAsTable("retail_org_dt.dim_products")